In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import math
from collections import namedtuple, deque
import random

In [29]:
win = 1.0
loss = -1.0
draw = -1.0

rows = 3
cols = 3

x = 1
o = 2
empty = 0


In [30]:
def get_move(model, board, player):
    board = board.flatten()
    player = torch.tensor([player])
    input_data = torch.cat((player, board))
    return model(input_data)

# this is the loss function for a tic tac toe model.
# predicition is the predicted best move of the model
# board is the current state that the board is in
# time is how many moves have been played
# winner is who ultimtely won the game

#CURRENT GOAL DONT PLACE IN SAME SPOT AS OTHER!!!!

filled_space_punishment = -10.0
resulted_in_loss_punishment = -1.0

def custom_loss(prediction, board, time, winner):
    prediction = prediction.flatten()
    board = board.flatten()
    player = x if time % 2 == 0 else o

    empty_slots_mask = (board != 0).float()
    loss = prediction * empty_slots_mask * filled_space_punishment

    max_value, max_index = torch.max(prediction, dim=0)

    if winner != player:
        loss[max_index] = resulted_in_loss_punishment * time

    return torch.mean(loss)
    

In [31]:
class TicTacToeNetworkA(nn.Module):
    def __init__(self):
        super(TicTacToeNetworkA, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(10, 8000),
            nn.ReLU(),
            nn.Linear(8000, 800),
            nn.ReLU(),
            nn.Linear(800, 9),
            nn.Softmax()
        )

    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.flatten(x)
        x = self.layers(x)
        return x
    
modelA = TicTacToeNetworkA()
optimizerA = optim.Adam(modelA.parameters(), lr=0.001)
criterionA = custom_loss

In [32]:
class TicTacToeNetworkB(nn.Module):
    def __init__(self):
        super(TicTacToeNetworkB, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(10, 8000),
            nn.ReLU(),
            nn.Linear(8000, 800),
            nn.ReLU(),
            nn.Linear(800, 9),
            nn.Softmax()
        )

    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.flatten(x)
        x = self.layers(x)
        return x
    
modelB = TicTacToeNetworkB()
optimizerB = optim.Adam(modelA.parameters(), lr=0.001, amsgrad=True)
criterionB = custom_loss

In [33]:
def user(dummy_var):
    while True:
        row = int(input(f'Enter row (0-{rows-1}): '))
        if row < rows and row >= 0:
            break

    while True:
        col = int(input(f'Enter col (0-{cols-1}): '))
        if col < cols and col >= 0:
            break

    board = torch.ones(rows,cols)
    board = board * empty

    board[row][col] = 1

    return board.view(1,9)

In [34]:
def checkForWin(board):
    for row in board:
        if torch.all(row == x):
            return x
        elif torch.all(row == o):
            return o

    for col in range(0, cols):
        col = board[:,col]
        if torch.all(col == x):
            return x
        elif torch.all(col == o):
            return o
        
    truth = board == o
    if truth[0][0] and truth[1][1] and truth[2][2]:
        return o
    elif truth[0][2] and truth[1][1] and truth[2][0]:
        return o
    
    truth = board == x
    if truth[0][0] and truth[1][1] and truth[2][2]:
        return x
    elif truth[0][2] and truth[1][1] and truth[2][0]:
        return x
        
    return empty

def print_tic_tac_toe_board(board):
    for row in range(3):
        # Print horizontal lines between rows
        if row != 0:
            print("-" * cols  * 3)

        for col in range(3):
            # Print vertical lines between columns
            if col != 0:
                print("|", end=" ")

            # Print the cell value
            if board[row][col] == 0:
                print(" ", end=" ")
            elif board[row][col] == 1:
                print("X", end=" ")
            elif board[row][col] == 2:
                print("O", end=" ")

        print()  # Move to the next line after printing each row
    print()



def run_game(playerA, playerB, show_game=False):
    game = torch.ones(rows*cols, rows,cols)
    game = game * empty

    for i in range(0, rows * cols):
        icon = empty
        
        if (i % 2) == 0:
            icon = x
            move = get_move(playerA, game[i], icon)
        else:
            icon = o
            move = get_move(playerB, game[i], icon)

        row, col = interprateMove(move)

        if game[i][row][col] == empty:
            for j in range(i,rows*cols):
                game[j][row][col] = icon
        else:
            if show_game: print("Unable to go there. Turn Lost")


        if show_game: print_tic_tac_toe_board(game[i])

        if checkForWin(game[i]):
            return checkForWin(game[i]), game
        
    return checkForWin(game[i]), game


def interprateMove(move):
    probs = F.softmax(move, dim=1)
    flat_indices = torch.argmax(probs.view(-1))
    original_indices = torch.unravel_index(flat_indices, move.shape)
    _, flat_index = original_indices
    row = math.floor(flat_index / rows)
    col = (flat_index % rows).item()
    return row, col



In [35]:
for i in range(0,100):
    models = (modelA, modelB)
    criterion = (criterionA, criterionB)
    optimizer = (optimizerA, optimizerB)
    
    # run an itteration of a game
    winner, game = run_game(models[0], models[1], True)

    #train the models on the game results
    for i, board in enumerate(game):
        player = x if i%2==0 else o
        # we can train them both on eachothers moves speeding up the learning time
        predictionA = get_move(models[0], board, player)
        predictionB = get_move(models[1], board, player)

        last_board = torch.ones(rows, cols) * empty
        
        if i != 0:
            last_board = game[i-1]

        lossA = criterion[0](predictionA, board=last_board, time=i, winner=winner)
        lossB = criterion[1](predictionB, board=last_board, time=i, winner=winner)
        optimizer[0].zero_grad()
        optimizer[1].zero_grad()
        lossA.backward()
        lossB.backward()
        optimizer[0].step()
        optimizer[1].step()

  |   | X 
---------
  |   |   
---------
  |   |   

  |   | X 
---------
O |   |   
---------
  |   |   

X |   | X 
---------
O |   |   
---------
  |   |   

X |   | X 
---------
O |   |   
---------
O |   |   

Unable to go there. Turn Lost
X |   | X 
---------
O |   |   
---------
O |   |   

X |   | X 
---------
O | O |   
---------
O |   |   

Unable to go there. Turn Lost
X |   | X 
---------
O | O |   
---------
O |   |   

Unable to go there. Turn Lost
X |   | X 
---------
O | O |   
---------
O |   |   

Unable to go there. Turn Lost
X |   | X 
---------
O | O |   
---------
O |   |   

  |   | X 
---------
  |   |   
---------
  |   |   

  |   | X 
---------
O |   |   
---------
  |   |   

Unable to go there. Turn Lost
  |   | X 
---------
O |   |   
---------
  |   |   

Unable to go there. Turn Lost
  |   | X 
---------
O |   |   
---------
  |   |   

Unable to go there. Turn Lost
  |   | X 
---------
O |   |   
---------
  |   |   

Unable to go there. Turn Lost
  | 

In [ ]:
winner,_ = run_game(user, modelB, True)

print(f"player {winner} is the winner!")



X |   |   
---------
  |   |   
---------
  |   |   

X |   |   
---------
  |   |   
---------
  | O |   

X |   |   
---------
  | X |   
---------
  | O |   

X |   |   
---------
  | X | O 
---------
  | O |   

X |   |   
---------
  | X | O 
---------
  | O | X 

player 1 is the winner!
